In [1]:
from ortools.sat.python import cp_model

Timeslots = [1,2,3]
Individuals = [i+1 for i in range(20)]
Teams = [team+1 for team in range(6)]

team_members = {
    1:[1, 6, 10, 11, 13, 15, 17, 19],
    2:[8, 10, 11, 14, 15],
    3:[2, 3, 5, 6, 14, 17, 20],
    4:[8, 14, 18, 19],
    5:[3, 4, 10, 11, 12, 17, 20],
    6:[1, 4, 7, 11, 12, 13, 17],
    }

In [2]:


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0


    def on_solution_callback(self):
        self.__solution_count += 1

    def solution_count(self):
        return self.__solution_count


def SearchForAllSolutionsSampleSat():
    """Showcases calling the solver to search for all solutions."""
    # Creates the model.
    model = cp_model.CpModel()
    
    x = {(team,t):model.NewBoolVar(f"x_{team}_{t}") for team in Teams for t in Timeslots}
    over_pt = {(p,t):model.NewIntVar(0,2, f"over_{p}_{t}") for p in Individuals for t in Timeslots}
    
    # Create the constraints.
    for team in Teams:
        model.AddExactlyOne([x[team,t] for t in Timeslots])

    for t in Timeslots:
        for p in Individuals:
            expressions = [x[team,t] for team in Teams if p in team_members[team] ]
            model.Add(sum(expressions)-1<= over_pt[p,t])            

    of_expr = sum([over_pt[p,t] for p in Individuals for t in Timeslots])
    model.Minimize(of_expr)
    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter([x,over_pt])
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = False
    # Solve.
    status = solver.Solve(model, solution_printer)
    print(f"Status = {solver.StatusName(status)}")
    
    print('Overassigned' , [i[0] for i in over_pt if solver.Value(over_pt[i]) ])


    team_assignment = {}    
    for (team,t),v in x.items():
        if solver.Value(v)>0:
            team_assignment[team,t] =1 
            print(f"Team {team} ------>  time {t}")
    print(f"Team member   {1} {2} {3} ")    

    for p in Individuals:
        time_assign = []
        for t in Timeslots:
            a = [team for team in Teams if p in team_members[team] and (team,t) in team_assignment]
            if len(a)>0:
                time_assign.append(len(a))
            else:
                time_assign.append(0)
        print(f"{p}             {time_assign[0]} {time_assign[1]} {time_assign[2]} ")    

SearchForAllSolutionsSampleSat()

Status = OPTIMAL
Overassigned [11, 14, 17]
Team 1 ------>  time 3
Team 2 ------>  time 1
Team 3 ------>  time 1
Team 4 ------>  time 2
Team 5 ------>  time 2
Team 6 ------>  time 1
Team member   1 2 3 
1             1 0 1 
2             1 0 0 
3             1 1 0 
4             1 1 0 
5             1 0 0 
6             1 0 1 
7             1 0 0 
8             1 1 0 
9             0 0 0 
10             1 1 1 
11             2 1 1 
12             1 1 0 
13             1 0 1 
14             2 1 0 
15             1 0 1 
16             0 0 0 
17             2 1 1 
18             0 1 0 
19             0 1 1 
20             1 1 0 
